In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
#import the libraries
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import openai
import io
from statistics import mean

## Load the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/MyDrive/challenge1_data.csv')

In [ ]:
df

,time_index,compound_id,step,braking_id,time,pressure_bar,temp_c,speed_kph,mu,A_1,C_1,B_1,C_2,F_1,B_2,E_1,B_3,B_4,F_2,F_3,B_5,C_3,F_4,F_5,B_6,B_7,B_8,A_2,A_3,B_9,B_10,C_4,B_11,B_12,E_2,E_3,E_4,C_5,B_13,F_6,C_6,B_14,B_15,E_5,B_16,A_4,F_7,F_8,E_6,E_7,B_17,B_18,B_19,E_8,F_9,B_20,F_10,B_21,A_5,E_9,F_11,F_12,E_10,B_22,D_1,E_11,B_23,B_24,A_6
0,1,17937-100,Burnishing 1,burnishing_1_1,0.865,8.553661,27.041474,73.433579,0.516617,0.0,5.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.2,0.0,0.0,0.0,0.0,30.1,1.4,0.0,0.0,7.0,0.0,10.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,8.0,0.0,5.0
1,2,17937-100,Burnishing 1,burnishing_1_1,0.955,8.674764,27.560890,72.141032,0.523087,0.0,5.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.2,0.0,0.0,0.0,0.0,30.1,1.4,0.0,0.0,7.0,0.0,10.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,8.0,0.0,5.0
2,3,17937-100,Burnishing 1,burnishing_1_1,1.045,8.541551,27.927537,70.848485,0.536882,0.0,5.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.2,0.0,0.0,0.0,0.0,30.1,1.4,0.0,0.0,7.0,0.0,10.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,8.0,0.0,5.0
3,4,17937-100,Burnishing 1,burnishing_1_1,1.135,8.571827,28.721938,69.580098,0.553362,0.0,5.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.2,0.0,0.0,0.0,0.0,30.1,1.4,0.0,0.0,7.0,0.0,10.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,8.0,0.0,5.0
4,5,17937-100,Burnishing 1,burnishing_1_1,1.225,8.426504,29.394123,68.311711,0.556536,0.0,5.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.2,0.0,0.0,0.0,0.0,30.1,1.4,0.0,0.0,7.0,0.0,10.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,8.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295423,3840,21963-13,Characteristic 7,characteristic_7_3,6.664,7.208340,212.851887,19.499339,0.608637,0.0,10.0,0.0,0.0,0.0,0.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.6,4.6,0.0,18.4,0.0,0.0,0.0,0.0,24.2,0.0,3.9,0.0,0.0,0.0,18.0,0.0,0.0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,2.6
1295424,3841,21963-13,Characteristic 7,characteristic_7_3,6.898,7.208340,215.644512,16.167709,0.595355,0.0,10.0,0.0,0.0,0.0,0.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.6,4.6,0.0,18.4,0.0,0.0,0.0,0.0,24.2,0.0,3.9,0.0,0.0,0.0,18.0,0.0,0.0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,2.6
1295425,3842,21963-13,Characteristic 7,characteristic_7_3,7.132,7.671138,218.278257,12.889230,0.579241,0.0,10.0,0.0,0.0,0.0,0.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.6,4.6,0.0,18.4,0.0,0.0,0.0,0.0,24.2,0.0,3.9,0.0,0.0,0.0,18.0,0.0,0.0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,2.6
1295426,3843,21963-13,Characteristic 7,characteristic_7_3,7.366,7.566123,220.527024,9.528608,0.581805,0.0,10.0,0.0,0.0,0.0,0.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.6,4.6,0.0,18.4,0.0,0.0,0.0,0.0,24.2,0.0,3.9,0.0,0.0,0.0,18.0,0.0,0.0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,2.6


# Fine tuning

In [ ]:
def get_composition(compound_id, braking_id, df):
  material_columns = ['A_1', 'C_1', 'B_1', 'C_2',
       'F_1', 'B_2', 'E_1', 'B_3', 'B_4', 'F_2', 'F_3', 'B_5', 'C_3', 'F_4',
       'F_5', 'B_6', 'B_7', 'B_8', 'A_2', 'A_3', 'B_9', 'B_10', 'C_4', 'B_11',
       'B_12', 'E_2', 'E_3', 'E_4', 'C_5', 'B_13', 'F_6', 'C_6', 'B_14',
       'B_15', 'E_5', 'B_16', 'A_4', 'F_7', 'F_8', 'E_6', 'E_7', 'B_17',
       'B_18', 'B_19', 'E_8', 'F_9', 'B_20', 'F_10', 'B_21', 'A_5', 'E_9',
       'F_11', 'F_12', 'E_10', 'B_22', 'D_1', 'E_11', 'B_23', 'B_24', 'A_6']
  composition_df = df[material_columns]
  composition = ""
  for index, row in composition_df.iterrows():
    for material_column in material_columns:
      if row[material_column] > 0:
        composition += str(row[material_column]) + "% " + material_column + ", "
    if composition[-1] == ' ':
      composition = composition[:-1]
    if composition[-1] == ',':
      composition = composition[:-1]
    break
  return composition

def get_user_prompt(compound_id, braking_id, df):
  material_composition = get_composition(compound_id, braking_id, df)
  prompt = "You have a compound with this composition of materials: " + material_composition + ". "
  prompt += "I want you to output the performance data which has the mu, pressure, time and speed values for a given time series. "
  prompt += "Remember that Mu is the most important value and should always follow the constraints and be as accurate as possible. "
  prompt += "Mu value should be approximately 0.6 with an allowed error of 0.1. "
  prompt += "What would be the performance data for this material with the above composition for the braking id '''" + braking_id + "'''?"
  return prompt

def get_assistant_prompt(compound_id, braking_id, df):
  prompt = ""
  prompt += "This is the performance data for braking id '''" + braking_id + "'''. "
  cols_to_measure = ['mu', 'time', 'pressure_bar', 'temp_c', 'speed_kph']
  new_df = df[cols_to_measure]
  new_df.rename(columns={"pressure_bar": "pressure", "speed_kph": "speed", "temp_c": "temp"})
  new_df = new_df.round(5)
  prompt += "This is the csv data: "
  prompt += new_df.to_csv(index=False)
  return prompt


def get_system_prompt():
  return "You are a highly skilled statistician from Harvard University who works at Brembo, where you specialize in performance braking systems and components as well as conducting research on braking systems. Given a friction material’s composition, you craft compelling synthetic performance data for a user given braking test type. The braking id will be delimited by triple quotes. You understand the importance of data analysis and seamlessly incorporate it for generating synthetic performance data based on historical performance data provided. You have a knack for paying attention to detail and curating synthetic data that is in line with the trends seen in the time series data you will be provided with. You are well versed with data and business analysis and use this knowledge for crafting the synthetic data."

In [ ]:
get_system_prompt()

'You are a highly skilled statistician from Harvard University who works at Brembo, where you specialize in performance braking systems and components as well as conducting research on braking systems. Given a friction material’s composition, you craft compelling synthetic performance data for a user given braking test type. The braking id will be delimited by triple quotes. You understand the importance of data analysis and seamlessly incorporate it for generating synthetic performance data based on historical performance data provided. You have a knack for paying attention to detail and curating synthetic data that is in line with the trends seen in the time series data you will be provided with. You are well versed with data and business analysis and use this knowledge for crafting the synthetic data.'

In [ ]:
brake_and_compund_tuples = pd.DataFrame(df.groupby(['braking_id', 'compound_id']))[0].unique()

In [ ]:
def get_fine_tune_example(df):
  i=0
  num_tuples = len(brake_and_compund_tuples)
  examples_list = []
  for braking_id, compound_id in brake_and_compund_tuples:
    if i>(0.05*num_tuples):
      break
    json_return = {}
    new_df = df.loc[(df['compound_id'] == compound_id) & (df['braking_id'] == braking_id)]
    json_return["messages"] = [{"role": "system", "content": get_system_prompt()}, {"role": "user", "content": get_user_prompt(compound_id, braking_id, new_df)}, {"role": "assistant", "content": get_assistant_prompt(compound_id, braking_id, new_df)}]
    if i%100 == 0:
      print("Done: " + str(i))
    examples_list.append(json_return)
    i+=1
  return examples_list

In [ ]:
num_tuples = len(brake_and_compund_tuples)

In [ ]:
num_tuples

41788

In [ ]:
i=0
examples = get_fine_tune_example(df)
for example in examples:
  if i>10:
    break
  print(example)
  i+=1
# get_fine_tune_example(df)

Done: 0
Done: 100
Done: 200
Done: 300
Done: 400
Done: 500
Done: 600
Done: 700
Done: 800
Done: 900
Done: 1000
Done: 1100
Done: 1200
Done: 1300
Done: 1400
Done: 1500
Done: 1600
Done: 1700
Done: 1800
Done: 1900
Done: 2000
{'messages': [{'role': 'system', 'content': 'You are a highly skilled statistician from Harvard University who works at Brembo, where you specialize in performance braking systems and components as well as conducting research on braking systems. Given a friction material’s composition, you craft compelling synthetic performance data for a user given braking test type. The braking id will be delimited by triple quotes. You understand the importance of data analysis and seamlessly incorporate it for generating synthetic performance data based on historical performance data provided. You have a knack for paying attention to detail and curating synthetic data that is in line with the trends seen in the time series data you will be provided with. You are well versed with data

In [ ]:
len(examples)

2090

In [ ]:
import json

In [ ]:
with open('data.jsonl', 'w') as f:
  for example in examples:
    json.dump(example, f)
    f.write("\n")

In [ ]:
file=open("data.jsonl", "rb"),

In [ ]:
jsonObj = pd.read_json(path_or_buf='data.jsonl', lines=True)

In [ ]:
jsonObj["messages"][0]

[{'role': 'system',
  'content': 'You are a highly skilled statistician from Harvard University who works at Brembo, where you specialize in performance braking systems and components as well as conducting research on braking systems. Given a friction material’s composition, you craft compelling synthetic performance data for a user given braking test type. The braking id will be delimited by triple quotes. You understand the importance of data analysis and seamlessly incorporate it for generating synthetic performance data based on historical performance data provided. You have a knack for paying attention to detail and curating synthetic data that is in line with the trends seen in the time series data you will be provided with. You are well versed with data and business analysis and use this knowledge for crafting the synthetic data.'},
 {'role': 'user',
  'content': "You have a compound with this composition of materials: 5.0% C_1, 7.5% E_1, 23.2% E_5, 30.1% E_6, 1.4% E_7, 7.0% B_1

## Testing

In [ ]:
df_test = pd.read_csv('drive/MyDrive/fully_constrained_new_chemical_compositions_10_more_3.csv')

In [ ]:
df_test

,E_6,E_5,E_1,F_9,F_2,C_1,B_21,B_19,B_23,A_6,E_9,B_7,B_14,B_4,F_8,D_1,E_10,C_2,F_12,B_10,B_2,E_7,B_17,B_8,B_12,C_3,B_6,A_1,E_11,E_4,F_6,F_7
0,26.9,20.6,9.9,8.4,7.4,6.9,4.4,3.0,2.9,2.6,1.4,1.4,1.3,1.3,1.2,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17.7,20.8,9.0,7.9,5.3,3.4,3.3,3.1,5.4,2.9,3.8,0.0,2.8,0.0,0.0,0.9,10.2,1.9,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26.6,20.7,8.6,11.0,4.7,6.1,5.0,4.8,4.2,2.0,1.7,0.9,0.0,0.0,0.0,0.6,0.0,0.0,0.0,1.2,1.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27.4,20.7,10.0,8.1,8.3,6.3,3.5,3.8,2.4,2.2,0.0,1.4,2.1,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,1.2,1.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20.4,21.1,10.5,8.4,7.7,2.4,3.3,4.2,5.8,3.0,2.1,0.0,1.7,0.0,0.0,0.6,6.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3,1.2,0.0,0.0,0.0,0.0,0.0
5,26.4,20.9,9.8,6.8,8.1,4.5,5.1,4.4,5.3,2.1,1.9,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,1.4,0.9,0.8,0.0,0.0
6,26.2,20.7,7.3,7.4,7.9,6.8,4.4,5.6,5.3,2.2,1.8,0.0,1.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0
7,27.2,20.9,8.9,6.9,6.4,7.0,4.8,3.4,2.7,1.7,2.0,0.0,2.4,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,1.4,0.0,0.0,0.0,0.0,0.0,2.8,0.0
8,20.4,19.6,13.4,14.3,1.1,4.3,3.4,3.5,2.9,3.2,2.7,0.0,2.2,0.0,0.0,0.7,0.0,4.2,0.0,0.0,0.0,0.0,3.0,0.0,1.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,24.1,20.3,9.7,6.5,0.0,6.5,3.4,4.6,5.2,2.5,3.7,0.0,1.8,0.0,0.0,0.8,3.5,0.0,0.0,0.0,0.0,0.0,2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.3,1.8


In [ ]:
def get_composition2(df):
  material_columns = ['E_6', 'E_5', 'E_1', 'F_9', 'F_2', 'C_1', 'B_21', 'B_19', 'B_23',
       'A_6', 'E_9', 'B_7', 'B_14', 'B_4', 'F_8', 'D_1', 'E_10', 'C_2',
       'F_12', 'B_10', 'B_2', 'E_7', 'B_17', 'B_8', 'B_12', 'C_3', 'B_6',
       'A_1', 'E_11', 'E_4', 'F_6', 'F_7']
  composition = ""
  for index, row in df.iterrows():
    for material_column in material_columns:
      if row[material_column] > 0:
        composition += str(row[material_column]) + "% " + material_column + ", "
    if composition[-1] == ' ':
      composition = composition[:-1]
    if composition[-1] == ',':
      composition = composition[:-1]
    break
  return composition

In [ ]:
pd.DataFrame(df_test.loc[0]).T

,E_6,E_5,E_1,F_9,F_2,C_1,B_21,B_19,B_23,A_6,E_9,B_7,B_14,B_4,F_8,D_1,E_10,C_2,F_12,B_10,B_2,E_7,B_17,B_8,B_12,C_3,B_6,A_1,E_11,E_4,F_6,F_7
0,26.9,20.6,9.9,8.4,7.4,6.9,4.4,3.0,2.9,2.6,1.4,1.4,1.3,1.3,1.2,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
get_user_prompt_model('burnishing_1_1')

"You have a compound with this composition of materials: 26.899999999999984% E_6, 20.6% E_5, 9.9% E_1, 8.4% F_9, 7.4% F_2, 6.9% C_1, 4.4% B_21, 3.0% B_19, 2.9% B_23, 2.6% A_6, 1.4% E_9, 1.4% B_7, 1.3% B_14, 1.3% B_4, 1.2% F_8, 0.3999999999999999% D_1. I want you to output the performance data which has the mu, pressure, time and speed values for a given time series. Remember that Mu is the most important value and should always follow the constraints and be as accurate as possible. Mu value should be approximately 0.6 with an allowed error of 0.1. What would be the performance data for this material with the above composition for the braking id '''burnishing_1_1'''?"

In [ ]:
csv_strs = []
for index, row in df_test.iterrows():
  openai.api_key = @key
  completion = openai.ChatCompletion.create(
    model="ft:gpt-3.5-turbo-0613:personal::89jkaokZ",
    messages=[
      {"role": "system", "content": get_system_prompt()},
      {"role": "user", "content": get_user_prompt_model('burnishing_1_1', row)}
    ]
  )
  csv_strs.append(completion.choices[0].message)

In [ ]:
for json in csv_strs:
  print(json.content.split("data: ",1)[1])

mu,time,pressure_bar,temp_c,speed_kph
0.44499,0.93,13.23095,27.75915,71.41826
0.44767,1.016,12.19282,28.93751,69.98207
0.46073,1.102,11.36769,30.37867,68.69718
0.46731,1.188,10.69927,32.10806,67.43675
0.47336,1.274,10.23488,33.8924,66.18069
0.49335,1.36,9.97334,35.9314,64.93611
0.50318,1.446,9.53031,38.24941,63.68005
0.51575,1.532,9.25164,40.58169,62.40042
0.51953,1.618,9.04387,43.03215,61.08659
0.51333,1.704,9.0708,45.42146,59.83173
0.53218,1.79,8.79729,47.83729,58.59299
0.54236,1.876,8.66402,50.19608,57.31813
0.53207,1.962,8.67068,52.55998,56.01972
0.55389,2.048,8.55187,54.89635,54.73564
0.55119,2.134,8.458,57.1156,53.4752
0.54656,2.22,8.46791,59.11284,52.18931
0.54834,2.306,8.53462,61.15998,50.92968
0.56042,2.392,8.4296,63.1562,49.63883
0.54271,2.478,8.37426,64.92832,48.37759
0.55091,2.564,8.3795,66.59556,47.11715
0.55422,2.65,8.42081,68.15381,45.83833
0.55289,2.736,8.38666,69.62651,44.58589
0.54743,2.822,8.4667,71.03098,43.31909
0.54861,2.908,8.49101,72.356,42.08881
0.53546,2.994,8

In [ ]:
# csv_str='mu,time,pressure_bar,temp_c,speed_kph\n0.41687,1.035,12.77897,25.76399,70.79642\n0.42998,1.11917,11.90548,26.70709,69.34141\n0.43375,1.20333,11.26851,27.77954,67.99757\n0.4406,1.2875,10.89369,28.99861,66.75089\n0.44994,1.37167,10.4442,30.24011,65.50139\n0.45393,1.45583,10.21695,31.48263,64.2519\n0.46474,1.54,10.04866,32.72514,63.0034\n0.46035,1.62417,10.0204,34.04712,61.72627\n0.46971,1.70833,9.80341,35.38951,60.47376\n0.46886,1.7925,9.91733,36.71149,59.20186\n0.49457,1.87667,9.67009,37.96724,57.90109\n0.49852,1.96083,9.49878,39.25357,56.59436\n0.49798,2.045,9.29662,40.62542,55.30532\n0.49398,2.12917,9.45299,41.89748,54.05783\n0.49401,2.21333,9.44491,43.25813,52.77989\n0.49583,2.2975,9.32475,44.60861,51.47518\n0.49124,2.38167,9.40944,45.90103,50.23151\n0.48924,2.46583,9.57975,47.2342,49.00789\n0.46972,2.55,9.55452,48.4248,47.71015\n0.47977,2.63417,9.68,49.79666,46.49841\n0.47515,2.71833,9.7143,51.07176,45.23885\n0.46037,2.8025,9.81322,52.34686,43.97331\n0.46329,2.88667,9.9387,53.58734,42.69739\n0.45615,2.97083,9.9992,54.76671,41.47779\n0.45034,3.055,10.1685,55.94813,40.20186\n0.45144,3.13917,10.17051,57.21099,38.98609\n0.44032,3.22333,10.31131,58.39445,37.68654\n0.43775,3.3075,10.26479,59.63493,36.46292\n0.43332,3.39167,10.43911,60.8062,35.19536\n0.43589,3.47583,10.5562,61.9581,33.98564\n0.41876,3.56,10.77421,63.0509,32.69861\n'

In [ ]:
data = csv_str
df = pd.DataFrame([x.split(';') for x in data.split('\n')])
print(df)

                                             0
0        mu,time,pressure_bar,temp_c,speed_kph
1     0.41687,1.035,12.77897,25.76399,70.79642
2   0.42998,1.11917,11.90548,26.70709,69.34141
3   0.43375,1.20333,11.26851,27.77954,67.99757
4     0.4406,1.2875,10.89369,28.99861,66.75089
5    0.44994,1.37167,10.4442,30.24011,65.50139
6    0.45393,1.45583,10.21695,31.48263,64.2519
7       0.46474,1.54,10.04866,32.72514,63.0034
8    0.46035,1.62417,10.0204,34.04712,61.72627
9    0.46971,1.70833,9.80341,35.38951,60.47376
10    0.46886,1.7925,9.91733,36.71149,59.20186
11   0.49457,1.87667,9.67009,37.96724,57.90109
12   0.49852,1.96083,9.49878,39.25357,56.59436
13     0.49798,2.045,9.29662,40.62542,55.30532
14   0.49398,2.12917,9.45299,41.89748,54.05783
15   0.49401,2.21333,9.44491,43.25813,52.77989
16    0.49583,2.2975,9.32475,44.60861,51.47518
17   0.49124,2.38167,9.40944,45.90103,50.23151
18    0.48924,2.46583,9.57975,47.2342,49.00789
19       0.46972,2.55,9.55452,48.4248,47.71015
20      0.479

In [ ]:
braking_ids = df['braking_id'].unique()
new_df = pd.DataFrame()
# new_df.columns = df.columns
for col in df.columns:
  print(col)
  new_df[col]=[]

# csv = ""
# csv_strs = []
# output_df = []
# for index, row in df_test.iterrows():
#   completion = openai.ChatCompletion.create(
#     model="ft:gpt-3.5-turbo-0613:personal::89jkaokZ",
#     messages=[
#       {"role": "system", "content": get_system_prompt()},
#       {"role": "user", "content": get_user_prompt_model('burnishing_1_1', row)}
#     ]
#   )


#   for json in csv_strs:
#     print(json.content.split("data: ",1)[1])
#   df = pd.DataFrame([x.split(';') for x in data.split('\n')])

#   csv_strs.append(completion.choices[0].message)

In [ ]:
df.columns.values

array(['time_index', 'compound_id', 'step', 'braking_id', 'time',
       'pressure_bar', 'temp_c', 'speed_kph', 'mu', 'A_1', 'C_1', 'B_1',
       'C_2', 'F_1', 'B_2', 'E_1', 'B_3', 'B_4', 'F_2', 'F_3', 'B_5',
       'C_3', 'F_4', 'F_5', 'B_6', 'B_7', 'B_8', 'A_2', 'A_3', 'B_9',
       'B_10', 'C_4', 'B_11', 'B_12', 'E_2', 'E_3', 'E_4', 'C_5', 'B_13',
       'F_6', 'C_6', 'B_14', 'B_15', 'E_5', 'B_16', 'A_4', 'F_7', 'F_8',
       'E_6', 'E_7', 'B_17', 'B_18', 'B_19', 'E_8', 'F_9', 'B_20', 'F_10',
       'B_21', 'A_5', 'E_9', 'F_11', 'F_12', 'E_10', 'B_22', 'D_1',
       'E_11', 'B_23', 'B_24', 'A_6'], dtype=object)

In [ ]:
new_df

,time_index,compound_id,step,braking_id,time,pressure_bar,temp_c,speed_kph,mu,A_1,C_1,B_1,C_2,F_1,B_2,E_1,B_3,B_4,F_2,F_3,B_5,C_3,F_4,F_5,B_6,B_7,B_8,A_2,A_3,B_9,B_10,C_4,B_11,B_12,E_2,E_3,E_4,C_5,B_13,F_6,C_6,B_14,B_15,E_5,B_16,A_4,F_7,F_8,E_6,E_7,B_17,B_18,B_19,E_8,F_9,B_20,F_10,B_21,A_5,E_9,F_11,F_12,E_10,B_22,D_1,E_11,B_23,B_24,A_6


In [ ]:
for json in csv_strs:
  print(json.content.split("data: ",1)[1])

In [ ]:
column_list = list(df.columns)


import csv
from io import StringIO


data_list = [json.content.split("data: ",1)[1] for json in csv_strs]
# Sample CSV string

def get_data_from_csv_string(csv_string):
  # Initialize an empty list to store the dictionaries
  data = []

  # Create a StringIO object to mimic a file
  csv_file = StringIO(csv_string)

  # Create a CSV reader that reads data into dictionaries
  csv_reader = csv.DictReader(csv_file)

  # Iterate over the rows in the CSV and convert them to dictionaries
  for row in csv_reader:
      data.append(row)
  return data

# Now, 'data' contains a list of dictionaries, with each dictionary representing a row from the CSV
# print(get_data_from_csv_string(data_list[0]))

dict_datas = [get_data_from_csv_string(item) for item in data_list]

print(len(dict_datas))



10


In [ ]:
df_test

,E_6,E_5,E_1,F_9,F_2,C_1,B_21,B_19,B_23,A_6,E_9,B_7,B_14,B_4,F_8,D_1,E_10,C_2,F_12,B_10,B_2,E_7,B_17,B_8,B_12,C_3,B_6,A_1,E_11,E_4,F_6,F_7
0,26.9,20.6,9.9,8.4,7.4,6.9,4.4,3.0,2.9,2.6,1.4,1.4,1.3,1.3,1.2,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17.7,20.8,9.0,7.9,5.3,3.4,3.3,3.1,5.4,2.9,3.8,0.0,2.8,0.0,0.0,0.9,10.2,1.9,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26.6,20.7,8.6,11.0,4.7,6.1,5.0,4.8,4.2,2.0,1.7,0.9,0.0,0.0,0.0,0.6,0.0,0.0,0.0,1.2,1.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27.4,20.7,10.0,8.1,8.3,6.3,3.5,3.8,2.4,2.2,0.0,1.4,2.1,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,1.2,1.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20.4,21.1,10.5,8.4,7.7,2.4,3.3,4.2,5.8,3.0,2.1,0.0,1.7,0.0,0.0,0.6,6.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3,1.2,0.0,0.0,0.0,0.0,0.0
5,26.4,20.9,9.8,6.8,8.1,4.5,5.1,4.4,5.3,2.1,1.9,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,1.4,0.9,0.8,0.0,0.0
6,26.2,20.7,7.3,7.4,7.9,6.8,4.4,5.6,5.3,2.2,1.8,0.0,1.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0
7,27.2,20.9,8.9,6.9,6.4,7.0,4.8,3.4,2.7,1.7,2.0,0.0,2.4,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,1.4,0.0,0.0,0.0,0.0,0.0,2.8,0.0
8,20.4,19.6,13.4,14.3,1.1,4.3,3.4,3.5,2.9,3.2,2.7,0.0,2.2,0.0,0.0,0.7,0.0,4.2,0.0,0.0,0.0,0.0,3.0,0.0,1.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,24.1,20.3,9.7,6.5,0.0,6.5,3.4,4.6,5.2,2.5,3.7,0.0,1.8,0.0,0.0,0.8,3.5,0.0,0.0,0.0,0.0,0.0,2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.3,1.8


In [ ]:
csv_string = ",".join(column_list) + "\n"
braking_id = "burnishing_1_1"

mat_columns = column_list[9:]
# ['time_index', 'compound_id', 'step', 'braking_id', 'time', 'pressure_bar', 'temp_c', 'speed_kph', 'mu', 'A_1', 'C_1', 'B_1', 'C_2', 'F_1', 'B_2', 'E_1', 'B_3', 'B_4', 'F_2', 'F_3', 'B_5', 'C_3', 'F_4', 'F_5', 'B_6', 'B_7', 'B_8', 'A_2', 'A_3', 'B_9', 'B_10', 'C_4', 'B_11', 'B_12', 'E_2', 'E_3', 'E_4', 'C_5', 'B_13', 'F_6', 'C_6', 'B_14', 'B_15', 'E_5', 'B_16', 'A_4', 'F_7', 'F_8', 'E_6', 'E_7', 'B_17', 'B_18', 'B_19', 'E_8', 'F_9', 'B_20', 'F_10', 'B_21', 'A_5', 'E_9', 'F_11', 'F_12', 'E_10', 'B_22', 'D_1', 'E_11', 'B_23', 'B_24', 'A_6']

for idx in range(10):
  dict_data = dict_datas[idx]
  for jdx in range(31):
    csv_string += f"{jdx + 1}, new-compound, Burnishing 1, {braking_id}, {dict_data[jdx]['time']}, {dict_data[jdx]['pressure_bar']}, {dict_data[jdx]['temp_c']}, {dict_data[jdx]['speed_kph']}, {dict_data[jdx]['mu']}"
    for mat_col in mat_columns:
      if mat_col in df_test.columns:
        csv_string += f", {df.iloc[idx][mat_col]}"
      else:
        csv_string += ", 0"
    csv_string += "\n"

# df_test.iloc[0]["E_6"]
# print(column_list)
# print(dict_datas[0][0])
# print(df_test.columns)
print(csv_string)

NameError: ignored

# **UTILS**

In [ ]:
# step_braking_id_map = {}
# for index, row in df[['step', 'braking_id']].iterrows():
#   step_braking_id_map[row['braking_id']] = row['step']
#   if index > 10000:
#     break
# print(step_braking_id_map)
step_braking_id_map = {'burnishing_1_1': 'Burnishing 1', 'burnishing_1_2': 'Burnishing 1', 'burnishing_1_3': 'Burnishing 1', 'burnishing_1_4': 'Burnishing 1', 'burnishing_1_5': 'Burnishing 1', 'burnishing_1_6': 'Burnishing 1', 'burnishing_1_7': 'Burnishing 1', 'burnishing_1_8': 'Burnishing 1', 'burnishing_1_9': 'Burnishing 1', 'burnishing_1_10': 'Burnishing 1', 'burnishing_1_11': 'Burnishing 1', 'burnishing_1_12': 'Burnishing 1', 'burnishing_1_13': 'Burnishing 1', 'burnishing_1_14': 'Burnishing 1', 'burnishing_1_15': 'Burnishing 1', 'burnishing_1_16': 'Burnishing 1', 'burnishing_1_17': 'Burnishing 1', 'characteristic_1_1': 'Characteristic 1', 'characteristic_1_2': 'Characteristic 1', 'characteristic_1_3': 'Characteristic 1', 'burnishing_2_1': 'Burnishing 2', 'burnishing_2_2': 'Burnishing 2', 'burnishing_2_3': 'Burnishing 2', 'burnishing_2_4': 'Burnishing 2', 'burnishing_2_5': 'Burnishing 2', 'characteristic_2_1': 'Characteristic 2', 'characteristic_2_2': 'Characteristic 2', 'characteristic_2_3': 'Characteristic 2', 'deceleration_series_1_1': 'Deceleration Series 1', 'deceleration_series_1_2': 'Deceleration Series 1', 'deceleration_series_1_3': 'Deceleration Series 1', 'deceleration_series_1_4': 'Deceleration Series 1', 'deceleration_series_1_5': 'Deceleration Series 1', 'deceleration_series_2_1': 'Deceleration Series 2', 'deceleration_series_2_2': 'Deceleration Series 2', 'deceleration_series_2_3': 'Deceleration Series 2', 'deceleration_series_2_4': 'Deceleration Series 2', 'deceleration_series_2_5': 'Deceleration Series 2', 'deceleration_series_3_1': 'Deceleration Series 3', 'deceleration_series_3_2': 'Deceleration Series 3', 'deceleration_series_3_3': 'Deceleration Series 3', 'deceleration_series_3_4': 'Deceleration Series 3', 'deceleration_series_3_5': 'Deceleration Series 3', 'deceleration_series_4_1': 'Deceleration Series 4', 'deceleration_series_4_2': 'Deceleration Series 4', 'deceleration_series_4_3': 'Deceleration Series 4', 'deceleration_series_4_4': 'Deceleration Series 4', 'deceleration_series_4_5': 'Deceleration Series 4', 'characteristic_3_1': 'Characteristic 3', 'characteristic_3_2': 'Characteristic 3', 'characteristic_3_3': 'Characteristic 3', 'fading_1_1': 'Fading 1', 'fading_1_2': 'Fading 1', 'fading_1_3': 'Fading 1', 'fading_1_4': 'Fading 1', 'fading_1_5': 'Fading 1', 'fading_1_6': 'Fading 1', 'fading_1_7': 'Fading 1', 'fading_1_8': 'Fading 1', 'fading_1_9': 'Fading 1', 'fading_1_10': 'Fading 1', 'fading_1_11': 'Fading 1', 'fading_1_12': 'Fading 1', 'fading_1_13': 'Fading 1', 'fading_1_14': 'Fading 1', 'fading_1_15': 'Fading 1', 'fading_1_16': 'Fading 1', 'fading_1_17': 'Fading 1', 'fading_1_18': 'Fading 1', 'fading_1_19': 'Fading 1', 'fading_1_20': 'Fading 1', 'characteristic_4_1': 'Characteristic 4', 'characteristic_4_2': 'Characteristic 4', 'characteristic_4_3': 'Characteristic 4', 'fading_2_1': 'Fading 2', 'fading_2_2': 'Fading 2', 'fading_2_3': 'Fading 2', 'fading_2_4': 'Fading 2', 'fading_2_5': 'Fading 2', 'fading_2_6': 'Fading 2', 'fading_2_7': 'Fading 2', 'fading_2_8': 'Fading 2', 'fading_2_9': 'Fading 2', 'fading_2_10': 'Fading 2', 'fading_2_11': 'Fading 2', 'fading_2_12': 'Fading 2', 'fading_2_13': 'Fading 2', 'fading_2_14': 'Fading 2', 'fading_2_15': 'Fading 2', 'fading_2_16': 'Fading 2', 'fading_2_17': 'Fading 2', 'fading_2_18': 'Fading 2', 'fading_2_19': 'Fading 2', 'fading_2_20': 'Fading 2', 'characteristic_5_1': 'Characteristic 5', 'characteristic_5_2': 'Characteristic 5', 'characteristic_5_3': 'Characteristic 5', 'increasing_temperature_1': 'Increasing temperature', 'increasing_temperature_2': 'Increasing temperature', 'increasing_temperature_3': 'Increasing temperature', 'increasing_temperature_4': 'Increasing temperature', 'increasing_temperature_5': 'Increasing temperature', 'increasing_temperature_6': 'Increasing temperature', 'increasing_temperature_7': 'Increasing temperature', 'increasing_temperature_8': 'Increasing temperature', 'deceleration_series_400_degrees_1': 'Deceleration Series 400 degrees', 'deceleration_series_400_degrees_2': 'Deceleration Series 400 degrees', 'deceleration_series_400_degrees_3': 'Deceleration Series 400 degrees', 'deceleration_series_400_degrees_4': 'Deceleration Series 400 degrees', 'deceleration_series_400_degrees_5': 'Deceleration Series 400 degrees', 'decreasing_temperature_1': 'Decreasing temperature', 'decreasing_temperature_2': 'Decreasing temperature', 'decreasing_temperature_3': 'Decreasing temperature', 'decreasing_temperature_4': 'Decreasing temperature', 'decreasing_temperature_5': 'Decreasing temperature', 'decreasing_temperature_6': 'Decreasing temperature', 'decreasing_temperature_7': 'Decreasing temperature', 'decreasing_temperature_8': 'Decreasing temperature', 'characteristic_6_1': 'Characteristic 6', 'characteristic_6_2': 'Characteristic 6', 'characteristic_6_3': 'Characteristic 6', 'characteristic_7_1': 'Characteristic 7', 'characteristic_7_2': 'Characteristic 7', 'characteristic_7_3': 'Characteristic 7'}

In [ ]:
def get_str(i):
  if i == 0:
    return "first"
  elif i == 1:
    return "second"
  elif i == 2:
    return "third"
  elif i == 3:
    return "fourth"
  elif i == 4:
    return "fifth"
  return "sixth"

cols_needed = ["mu", "time", "pressure_bar", "temp_c", "speed_kph"]
def get_perf_data_from_compound_id(compound_id, braking_id):
  ret = {}
  df_temp = pd.DataFrame(df[(df.compound_id==compound_id) & (df.braking_id==braking_id)])[cols_needed]
  return df_temp.round(3).to_csv(index=False)

def get_user_prompt_knn_context(compound_ids, braking_id):
  prompt = "Here are some similar compounds and their performance data you have to use for reference. "
  prompt += "Use them for generating the performance data. "

  i=0
  for compound_id in compound_ids:
    material_csv = get_perf_data_from_compound_id(compound_id, braking_id)
    prompt += "\n\nThe time series data for braking id " + braking_id + "  of the " + get_str(i) + " most similar compound is this:\n"
    prompt += get_perf_data_from_compound_id(compound_id, braking_id)
    i += 1

  prompt += "Notice that the value "
  prompt += "of mu is approximately 0.6. Strictly ensure that your output performance data "
  prompt += "also has the value of mu around 0.6 for all the time."
  return prompt

In [ ]:
def get_user_prompt_model(braking_id, row):
  compound_ids = row.neighbours.split("$")[0:4]
  material_composition = get_composition2(pd.DataFrame(row).T)
  prompt = "You have a compound with this composition of materials: " + material_composition + ". "
  prompt += "I want you to output the performance data which has the mu, pressure, time and speed values for a given time series. "
  prompt += get_user_prompt_knn_context(compound_ids, braking_id)
  prompt += "Remember that Mu is the most important value and should always follow the constraints and be as accurate as possible. "
  prompt += "Make sure that your average Mu is the same as the average Mu in above time series. Any mistake here is intolerable. "
  prompt += "Strictly ensure that you only generate 31 datapoints in the time series! Do not ever miss this. "
  prompt += "What would be the performance data for this material with the above composition for the braking id '''" + braking_id + "'''?"
  return prompt

In [ ]:
def get_column_names():
  return ",".join(list(df.columns)) + "\n"

import csv
import random
from io import StringIO

# Sample CSV string
def get_data_from_csv_string(csv_string):
  # Initialize an empty list to store the dictionaries
  data = []
  # Create a StringIO object to mimic a file
  csv_file = StringIO(csv_string)
  # Create a CSV reader that reads data into dictionaries
  csv_reader = csv.DictReader(csv_file)
  # Iterate over the rows in the CSV and convert them to dictionaries
  idx = 0
  for row in csv_reader:
    data.append(row)
    if idx == 0:
      columns_match = ['time', 'pressure_bar', 'temp_c', 'speed_kph', 'mu']
      columns = row.keys()
      assert set(columns_match) == set(columns)
      idx = 1

  return data
# Now, 'data' contains a list of dictionaries, with each dictionary representing a row from the CSV
# print(get_data_from_csv_string(data_list[0]))

def add_rows(compound_id, braking_id, csv_strs, df_test, row_index, start_idx):
  csv_string = ""
  data_list = [json.content.split(":",1)[1].strip() for json in csv_strs]
  dict_datas = [get_data_from_csv_string(item) for item in data_list]
  column_list = list(df.columns)
  mat_columns = column_list[9:]
  # ['time_index', 'compound_id', 'step', 'braking_id', 'time', 'pressure_bar', 'temp_c', 'speed_kph', 'mu', 'A_1', 'C_1', 'B_1', 'C_2', 'F_1', 'B_2', 'E_1', 'B_3', 'B_4', 'F_2', 'F_3', 'B_5', 'C_3', 'F_4', 'F_5', 'B_6', 'B_7', 'B_8', 'A_2', 'A_3', 'B_9', 'B_10', 'C_4', 'B_11', 'B_12', 'E_2', 'E_3', 'E_4', 'C_5', 'B_13', 'F_6', 'C_6', 'B_14', 'B_15', 'E_5', 'B_16', 'A_4', 'F_7', 'F_8', 'E_6', 'E_7', 'B_17', 'B_18', 'B_19', 'E_8', 'F_9', 'B_20', 'F_10', 'B_21', 'A_5', 'E_9', 'F_11', 'F_12', 'E_10', 'B_22', 'D_1', 'E_11', 'B_23', 'B_24', 'A_6']

  for idx in range(len(csv_strs)):
    dict_data = dict_datas[idx]
    assert len(dict_data) >= 31

    for jdx in range(31):
      csv_string += f"{start_idx + jdx + 1}, new-compound{compound_id}, {step_braking_id_map[braking_id]}, {braking_id}, {dict_data[jdx]['time']}, {dict_data[jdx]['pressure_bar']}, {dict_data[jdx]['temp_c']}, {dict_data[jdx]['speed_kph']}, {dict_data[jdx]['mu']}"
      for mat_col in mat_columns:
        if mat_col in df_test.columns:
          csv_string += f", {df_test.iloc[row_index][mat_col]}"
        else:
          csv_string += ", 0"
      csv_string += "\n"
  return csv_string

def get_avg_mu(csv_strs):
  data_list = [json.content.split(":",1)[1].strip() for json in csv_strs]
  dict_datas = [get_data_from_csv_string(item) for item in data_list]
  for idx in range(len(csv_strs)):
    dict_data = dict_datas[idx]
    mus = [float(dict_data[j]['mu']) for j in range(31)]
    return mean(mus)

def response_validate(csv_strs):
  split_str = csv_strs[0].content.split(":",1)
  if len(split_str) < 2:
    print("Problem with splitting " + csv_strs[0].content)
    return False
  data_list = [split_str[1].strip() for json in csv_strs]
  dict_datas = [get_data_from_csv_string(item) for item in data_list]
  for idx in range(len(csv_strs)):
    dict_data = dict_datas[idx]
    if len(dict_data) < 31:
      print("Data len: " + str(len(dict_data)))
      # for i in range(len(dict_data)):
        # print(dict_data[i]['time'] + " \t " + dict_data[i]['mu'] + " \t " + dict_data[i]['speed_kph'] + " \t " + dict_data[i]['temp_c'] + " \t " + dict_data[i]['pressure_bar'])
      return False
  return True

def get_me_a_response(braking_id, row):
  openai.api_key = @key
  completion = openai.ChatCompletion.create(
    model="ft:gpt-3.5-turbo-0613:personal::89jkaokZ",
    messages=[
      {"role": "system", "content": get_system_prompt()},
      {"role": "user", "content": get_user_prompt_model(braking_id, row)}
    ]
  )
  responses.append(completion.choices[0].message)
  csv_strs = [completion.choices[0].message]
  retries = 0
  while response_validate(csv_strs) == False:
    completion = openai.ChatCompletion.create(
      model="ft:gpt-3.5-turbo-0613:personal::89jkaokZ",
      messages=[
        {"role": "system", "content": get_system_prompt()},
        {"role": "user", "content": get_user_prompt_model(braking_id, row)}
        ]
      )
    csv_strs = [completion.choices[0].message]
    retries += 1
    print("Retryingggg " + str(retries) + " braking id " + braking_id)
    if retries == 10:
      break
  return csv_strs

responses = []
def call_model_for_dataset(dataset_path):
  df_test = pd.read_csv(dataset_path)
  csv_string = ""
  for index, row in df_test.iterrows():  # Iterate over compounds
    compound_id = random.randint(0,10000000)
    print("Iterating compound " + str(index))
    tmp = 0
    start_idx = 0
    for braking_id in step_braking_id_map.keys():
      csv_strs = get_me_a_response(braking_id, row)
      print("Mean is:" + str(get_avg_mu(csv_strs)))
      csv_string += add_rows(compound_id, braking_id, csv_strs, df_test, index, start_idx)
      start_idx += 31
      print("Called model for compound: " + str(index) + " braking id: " + braking_id)
      tmp += 1
      # if tmp == 5:
        # break
    if index == 0:
      break
  return csv_string


In [ ]:
csv_string = ""

csv_string += get_column_names()
csv_string += call_model_for_dataset('drive/MyDrive/materials_with_neighbours.csv')

print(csv_string)

Iterating compound 0
Mean is:0.4950196774193548
Called model for compound: 0 braking id: burnishing_1_1
Mean is:0.45303225806451614
Called model for compound: 0 braking id: burnishing_1_2
Mean is:0.46119354838709675
Called model for compound: 0 braking id: burnishing_1_3
Mean is:0.42043870967741936
Called model for compound: 0 braking id: burnishing_1_4
Mean is:0.39335709677419356
Called model for compound: 0 braking id: burnishing_1_5
Mean is:0.44403258064516127
Called model for compound: 0 braking id: burnishing_1_6
Mean is:0.500741935483871
Called model for compound: 0 braking id: burnishing_1_7
Mean is:0.4966451612903226
Called model for compound: 0 braking id: burnishing_1_8
Mean is:0.46333032258064516
Called model for compound: 0 braking id: burnishing_1_9
Mean is:0.44762870967741936
Called model for compound: 0 braking id: burnishing_1_10
Mean is:0.4549412903225806
Called model for compound: 0 braking id: burnishing_1_11
Mean is:0.5128064516129033
Called model for compound: 0 br

In [ ]:
def get_average_mus(compound_id, df):
  for braking_id in step_braking_id_map.keys():
    print(braking_id, df[(df.compound_id==compound_id) & (df.braking_id==braking_id)].mu.mean())

# def get_average_mus(compound_id1, df1, compound_id2, df2):
#   for braking_id in step_braking_id_map.keys():
#     print(braking_id, df1[(df1.compound_id==compound_id1) & (df1.braking_id==braking_id)].mu.mean(), df2[(df2.compound_id==compound_id2) & (df2.braking_id==braking_id)].mu.mean())

In [ ]:
print(df[(df.compound_id=="17937-261")].mu.mean())
get_average_mus("17937-261", df, "17937-203", df)

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
df2.braking_id = df2.braking_id.apply(str)

In [ ]:
df2

,time_index,compound_id,step,braking_id,time,pressure_bar,temp_c,speed_kph,mu,A_1,C_1,B_1,C_2,F_1,B_2,E_1,B_3,B_4,F_2,F_3,B_5,C_3,F_4,F_5,B_6,B_7,B_8,A_2,A_3,B_9,B_10,C_4,B_11,B_12,E_2,E_3,E_4,C_5,B_13,F_6,C_6,B_14,B_15,E_5,B_16,A_4,F_7,F_8,E_6,E_7,B_17,B_18,B_19,E_8,F_9,B_20,F_10,B_21,A_5,E_9,F_11,F_12,E_10,B_22,D_1,E_11,B_23,B_24,A_6
0,1,new-compound6642218,Burnishing 1,burnishing_1_1,0.98000,12.40945,26.11499,73.04625,0.45054,0.0,6.9,0.0,0.0,0.0,0.0,9.9,0.0,1.3,7.4,0,0.0,0.0,0,0.0,0.0,1.4,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,1.3,0.0,20.6,0,0,0.0,1.2,26.9,0.0,0.0,0,3.0,0,8.4,0,0,4.4,0.0,1.4,0.0,0.0,0.0,0,0.4,0.0,2.9,0,2.6
1,2,new-compound6642218,Burnishing 1,burnishing_1_1,1.06667,10.24082,26.68123,71.5433,0.46857,0.0,6.9,0.0,0.0,0.0,0.0,9.9,0.0,1.3,7.4,0,0.0,0.0,0,0.0,0.0,1.4,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,1.3,0.0,20.6,0,0,0.0,1.2,26.9,0.0,0.0,0,3.0,0,8.4,0,0,4.4,0.0,1.4,0.0,0.0,0.0,0,0.4,0.0,2.9,0,2.6
2,3,new-compound6642218,Burnishing 1,burnishing_1_1,1.15333,9.45748,27.25665,70.2565,0.48360,0.0,6.9,0.0,0.0,0.0,0.0,9.9,0.0,1.3,7.4,0,0.0,0.0,0,0.0,0.0,1.4,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,1.3,0.0,20.6,0,0,0.0,1.2,26.9,0.0,0.0,0,3.0,0,8.4,0,0,4.4,0.0,1.4,0.0,0.0,0.0,0,0.4,0.0,2.9,0,2.6
3,4,new-compound6642218,Burnishing 1,burnishing_1_1,1.24000,9.11408,27.94104,68.97389,0.49137,0.0,6.9,0.0,0.0,0.0,0.0,9.9,0.0,1.3,7.4,0,0.0,0.0,0,0.0,0.0,1.4,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,1.3,0.0,20.6,0,0,0.0,1.2,26.9,0.0,0.0,0,3.0,0,8.4,0,0,4.4,0.0,1.4,0.0,0.0,0.0,0,0.4,0.0,2.9,0,2.6
4,5,new-compound6642218,Burnishing 1,burnishing_1_1,1.32667,8.88893,28.56680,67.75662,0.49898,0.0,6.9,0.0,0.0,0.0,0.0,9.9,0.0,1.3,7.4,0,0.0,0.0,0,0.0,0.0,1.4,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,1.3,0.0,20.6,0,0,0.0,1.2,26.9,0.0,0.0,0,3.0,0,8.4,0,0,4.4,0.0,1.4,0.0,0.0,0.0,0,0.4,0.0,2.9,0,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3839,3840,new-compound6642218,Characteristic 7,characteristic_7_3,5.84300,7.15990,156.48640,43.60848,0.62187,0.0,6.9,0.0,0.0,0.0,0.0,9.9,0.0,1.3,7.4,0,0.0,0.0,0,0.0,0.0,1.4,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,1.3,0.0,20.6,0,0,0.0,1.2,26.9,0.0,0.0,0,3.0,0,8.4,0,0,4.4,0.0,1.4,0.0,0.0,0.0,0,0.4,0.0,2.9,0,2.6
3840,3841,new-compound6642218,Characteristic 7,characteristic_7_3,6.04500,7.45518,158.66905,41.21403,0.60112,0.0,6.9,0.0,0.0,0.0,0.0,9.9,0.0,1.3,7.4,0,0.0,0.0,0,0.0,0.0,1.4,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,1.3,0.0,20.6,0,0,0.0,1.2,26.9,0.0,0.0,0,3.0,0,8.4,0,0,4.4,0.0,1.4,0.0,0.0,0.0,0,0.4,0.0,2.9,0,2.6
3841,3842,new-compound6642218,Characteristic 7,characteristic_7_3,6.24700,7.63753,160.66727,38.89034,0.59866,0.0,6.9,0.0,0.0,0.0,0.0,9.9,0.0,1.3,7.4,0,0.0,0.0,0,0.0,0.0,1.4,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,1.3,0.0,20.6,0,0,0.0,1.2,26.9,0.0,0.0,0,3.0,0,8.4,0,0,4.4,0.0,1.4,0.0,0.0,0.0,0,0.4,0.0,2.9,0,2.6
3842,3843,new-compound6642218,Characteristic 7,characteristic_7_3,6.44900,7.73810,162.70626,36.51148,0.57668,0.0,6.9,0.0,0.0,0.0,0.0,9.9,0.0,1.3,7.4,0,0.0,0.0,0,0.0,0.0,1.4,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,1.3,0.0,20.6,0,0,0.0,1.2,26.9,0.0,0.0,0,3.0,0,8.4,0,0,4.4,0.0,1.4,0.0,0.0,0.0,0,0.4,0.0,2.9,0,2.6


In [ ]:
df2.mu.dtype

dtype('float64')

In [ ]:
df2['braking_id'] = df2['braking_id'].astype(str)

In [ ]:
df2['braking_id'].dtype

dtype('O')

In [ ]:
df2[df2.braking_id=="burnishing_1_1"]

,time_index,compound_id,step,braking_id,time,pressure_bar,temp_c,speed_kph,mu,A_1,C_1,B_1,C_2,F_1,B_2,E_1,B_3,B_4,F_2,F_3,B_5,C_3,F_4,F_5,B_6,B_7,B_8,A_2,A_3,B_9,B_10,C_4,B_11,B_12,E_2,E_3,E_4,C_5,B_13,F_6,C_6,B_14,B_15,E_5,B_16,A_4,F_7,F_8,E_6,E_7,B_17,B_18,B_19,E_8,F_9,B_20,F_10,B_21,A_5,E_9,F_11,F_12,E_10,B_22,D_1,E_11,B_23,B_24,A_6


In [ ]:
df2.to_csv(filepath)

In [ ]:
get_average_mus('new-compound6642218', df2)

burnishing_1_1 nan
burnishing_1_2 nan
burnishing_1_3 nan
burnishing_1_4 nan
burnishing_1_5 nan
burnishing_1_6 nan
burnishing_1_7 nan
burnishing_1_8 nan
burnishing_1_9 nan
burnishing_1_10 nan
burnishing_1_11 nan
burnishing_1_12 nan
burnishing_1_13 nan
burnishing_1_14 nan
burnishing_1_15 nan
burnishing_1_16 nan
burnishing_1_17 nan
characteristic_1_1 nan
characteristic_1_2 nan
characteristic_1_3 nan
burnishing_2_1 nan
burnishing_2_2 nan
burnishing_2_3 nan
burnishing_2_4 nan
burnishing_2_5 nan
characteristic_2_1 nan
characteristic_2_2 nan
characteristic_2_3 nan
deceleration_series_1_1 nan
deceleration_series_1_2 nan
deceleration_series_1_3 nan
deceleration_series_1_4 nan
deceleration_series_1_5 nan
deceleration_series_2_1 nan
deceleration_series_2_2 nan
deceleration_series_2_3 nan
deceleration_series_2_4 nan
deceleration_series_2_5 nan
deceleration_series_3_1 nan
deceleration_series_3_2 nan
deceleration_series_3_3 nan
deceleration_series_3_4 nan
deceleration_series_3_5 nan
deceleration_ser

In [ ]:
type(df2.braking_id)

pandas.core.series.Series

In [ ]:
df2.astype('int32').dtypes

AttributeError: ignored

In [ ]:
# print(csv_string)
# data = csv_string
# df = pd.DataFrame([x.split(',') for x in data.split('\n')])
df2 = pd.read_csv(io.StringIO(csv_string))
df2

for braking_id in step_braking_id_map.keys():
  normal_avg = df[df.braking_id==braking_id].mu.mean()
  new_avg = df2[df2.braking_id==braking_id].mu.mean()
  print(braking_id + ": " + str(normal_avg) + " " + str(new_avg))
# get_average_mus("17937-261", df, "new-compound3807681", df2)
# print(df2.groupby(['braking_id']).mu.mean())

burnishing_1_1: 0.46639105865338226 nan
burnishing_1_2: 0.4500705987744121 nan
burnishing_1_3: 0.4433398696071779 nan
burnishing_1_4: 0.44841403916210193 nan
burnishing_1_5: 0.45550634912900045 nan
burnishing_1_6: 0.46232825074876754 nan
burnishing_1_7: 0.46809815896616874 nan
burnishing_1_8: 0.4732740603915371 nan
burnishing_1_9: 0.4779558480531196 nan
burnishing_1_10: 0.48201762728113146 nan
burnishing_1_11: 0.519653013048154 nan
burnishing_1_12: 0.5235341537823827 nan
burnishing_1_13: 0.5136161038531487 nan
burnishing_1_14: 0.5181054596585166 nan
burnishing_1_15: 0.5248609331526067 nan
burnishing_1_16: 0.5300323848992508 nan
burnishing_1_17: 0.5371628988479423 nan
characteristic_1_1: 0.5431747388708484 nan
characteristic_1_2: 0.5677718208567253 nan
characteristic_1_3: 0.5898522079466794 nan
burnishing_2_1: 0.6003295369976565 nan
burnishing_2_2: 0.6642162495483825 nan
burnishing_2_3: 0.6956732131717731 nan
burnishing_2_4: 0.7085432385479804 nan
burnishing_2_5: 0.708151091135258 nan
c

In [ ]:
print(df2.groupby('braking_id').mu.mean())
# print(df.groupby('braking_id').mu.mean())

braking_id
 burnishing_1_1                       0.495020
 burnishing_1_10                      0.447629
 burnishing_1_11                      0.454941
 burnishing_1_12                      0.512806
 burnishing_1_13                      0.518451
 burnishing_1_14                      0.504941
 burnishing_1_15                      0.488419
 burnishing_1_16                      0.506064
 burnishing_1_17                      0.484871
 burnishing_1_2                       0.453032
 burnishing_1_3                       0.461194
 burnishing_1_4                       0.420439
 burnishing_1_5                       0.393357
 burnishing_1_6                       0.444033
 burnishing_1_7                       0.500742
 burnishing_1_8                       0.496645
 burnishing_1_9                       0.463330
 burnishing_2_1                       0.498944
 burnishing_2_2                       0.624368
 burnishing_2_3                       0.515662
 burnishing_2_4                       0.695998
 b

In [ ]:
df2.to_csv('test.csv', index=False)